In [1]:
# add parent dir to sys path for import of modules
import os
import sys

# find recursively the project root dir
parent_dir = str(os.getcwdb())
while not os.path.exists(os.path.join(parent_dir, "README.md")):
    parent_dir = os.path.abspath(os.path.join(parent_dir, os.pardir))
sys.path.insert(0, parent_dir)

C:\Users\janek\Development\Git\master-thesis


In [43]:
import json
import pandas as pd

from utils import ROOT_DIR, load_pickle
from PetReader import pet_reader

In [65]:
with open(os.path.join(ROOT_DIR, "data/results/keywords_filtered/key_words_literature_tc_filtered_og_syn/filtering_XOR Gateways.json"), "r") as file:
    filtering_file = json.load(file)

In [71]:
KEPT = 'kept'
CORRECTLY_KEPT = 'correctly_kept'
WRONGLY_KEPT = 'wrongly_kept'
DROPPED = 'dropped'
CORRECTLY_DROPPED = 'correctly_dropped'
WRONGLY_DROPPED = 'wrongly_dropped'

statistics = {}

for doc_name, filterings in filtering_file.items():
    
    if doc_name != 'doc-2.2':
        continue
    
    print(doc_name)
    doc_gold = pet_reader.relations_dataset.GetSentencesWithIdsAndNerTagLabels(pet_reader.get_document_number(doc_name))
    
    for sentence_idx, tokens in enumerate(doc_gold):
        sentence_filterings = [f for f in filterings if f[0] == sentence_idx]
        for f in sentence_filterings:
            t = f[1][0].lower()
            if t not in statistics:
                statistics[t] = {KEPT: 0, CORRECTLY_KEPT: 0, WRONGLY_KEPT: 0, DROPPED: 0, CORRECTLY_DROPPED: 0, WRONGLY_DROPPED: 0}
            print(' '.join([t[0] for t in tokens]))
            # tuple in token
            if tuple(f[1]) in tokens:
                if f[3] == 'kept':
                    statistics[t][KEPT] += 1
                    statistics[t][CORRECTLY_KEPT] += 1
                    print(sentence_idx, tokens[f[1][1]])
                    print(f, "CORRECTLY_KEPT\n")
#                     if t == 'or':
#                         print(sentence_idx, tokens)
#                         print(f, "CORRECTLY_KEPT")
                else:
                    statistics[t][DROPPED] += 1
                    statistics[t][WRONGLY_DROPPED] += 1
                    print(sentence_idx, tokens[f[1][1]])
                    print(f, "WRONGLY_DROPPED\n")
            # tuple not in token
            else:
                if f[3] == 'dropped':
                    statistics[t][DROPPED] += 1
                    statistics[t][CORRECTLY_DROPPED] += 1
                    print(sentence_idx, tokens[f[1][1]])
                    print(f, "CORRECTLY_DROPPED\n")
                else:
                    statistics[t][KEPT] += 1
                    statistics[t][WRONGLY_KEPT] += 1
                    print(sentence_idx, tokens[f[1][1]])
                    print(f, "WRONGLY_KEPT\n")


doc-2.2
In case of any errors these should be corrected on the basis of a simple error list .
5 ('In', 0, 'B-XOR Gateway')
[5, ['In', 0, 'B-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

In case of any errors these should be corrected on the basis of a simple error list .
5 ('case', 1, 'I-XOR Gateway')
[5, ['case', 1, 'I-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

In case of any errors these should be corrected on the basis of a simple error list .
5 ('of', 2, 'I-XOR Gateway')
[5, ['of', 2, 'I-XOR Gateway'], 6, 'dropped'] WRONGLY_DROPPED

In case the customer does not exist in the customer data base , a new customer object is being created which will remain the data object of interest during the rest of the process flow .
7 ('In', 0, 'B-XOR Gateway')
[7, ['In', 0, 'B-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

In case the customer does not exist in the customer data base , a new customer object is being created which will remain the data object of interest during the rest of the process flow .


In [63]:
rows = []
for keyword, stats in statistics.items():
    print(stats)
    kept_ratio = stats[KEPT] / (stats[KEPT] + stats[DROPPED])
    dropped_ratio = stats[DROPPED] / (stats[KEPT] + stats[DROPPED])
    stats["kept/dropped"] = f"{round(kept_ratio, 2)}/{round(dropped_ratio, 2)}"
    print(keyword, stats)
    rows.append({**{"word": keyword}, **stats})

{'kept': 5, 'correctly_kept': 1, 'wrongly_kept': 4, 'dropped': 59, 'correctly_dropped': 40, 'wrongly_dropped': 19}
or {'kept': 5, 'correctly_kept': 1, 'wrongly_kept': 4, 'dropped': 59, 'correctly_dropped': 40, 'wrongly_dropped': 19, 'kept/dropped': '0.08/0.92'}


In [52]:
df = pd.DataFrame.from_dict(rows)
df.head(10)
df.to_excel(os.path.join(ROOT_DIR, "data/paper_stats/token_cls/filtering_stats.xlsx"), index=False)